In [39]:
# -------------------------------------
# Master script for Maritime Risk Model
#
#         Ryan Lloyd
#         22/10/2018
# -------------------------------------

# ------- USAGE --------
#
# 1. Requires list of AIS data and incidents
# 2. Requires some static model parameters
#
# ----------------------

In [40]:
# Load environments
import os
import sys
import arcgis
import arcpy
import pandas
#import geopandas
import csv
import numpy
import matplotlib
import matplotlib.pyplot
import pylab
import glob
import zipfile
from arcpy.sa import *
from importlib import reload

In [41]:
# Input files
AIS123_root= r"C:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/risk_data/AIS_123/" # OS SPECIFIC "/"
AIS005_root= r"C:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/risk_data/AIS_005/" # OS SPECIFIC
incidents_root= r"C:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/risk_data/incidents/" # OS SPECIFIC
AIS123_file = "files_123.txt"
#AIS123_file = "Feb05_2017.txt"
AIS005_file = "files_005.txt"
incidents_file = "incidents.txt"
continent_polygon = "c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/risk_data/continents/World_Continents.lyrx"

FileGDB_path = "c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/working_dir/"
FileGDB_name = "risk_model_5-7.gdb"

In [ ]:
# NEW GDBs for each cell shape? Incorporate into name?
# If auto name = true, else 

In [59]:
# Variables (FN size etc)

spatial_ref = arcpy.SpatialReference(4326) #GC_WGS_1984 - 4326, WC - 3857
AISjoin_field = "MMSI"
AISjoin_order = "ISODate"
grid_extent = [7, 10, -3, -7] # N,E,S,W
SIZE  = 2500   # AREA (km^2) of grid
SHAPE = "SQUARE" # Shape of grid cells

In [43]:
# ------ DO NOT EDIT BELOW HERE ----------

In [44]:
# Add folder of python modules:
sys.path.append(r"C:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/risk_functions/")

In [56]:
import load_data
import create_FileGDB
import traffic
import incidents
import inc_frequency
reload(inc_frequency)
reload(incidents)
reload(load_data)
reload(create_FileGDB)
reload(traffic)

<module 'traffic' from 'C:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/risk_functions\\traffic.py'>

In [57]:
# Load local modules
from load_data import csv2shp, loadais, loadincidents, csv2dict
from create_FileGDB import creategdb
from traffic import ais2traffic
from create_grid import tes_grid
from incidents import incident_density
from traffic import traffic_density
from inc_frequency import inc_freq

In [47]:
# Set up FileGDB
creategdb(FileGDB_path, FileGDB_name)
arcpy.env.workspace = FileGDB_path + FileGDB_name

FileGDB already exists


In [48]:
# 0948

In [49]:
# Load in the AIS and incidents
incidentspts = csv2shp(incidents_root,incidents_file,FileGDB_path,FileGDB_name)
AIS123pts = csv2shp(AIS123_root, AIS123_file,FileGDB_path,FileGDB_name)

csv files converted to points:
incidents_050217.csv >> incidents_050217
incidents_060217.csv >> incidents_060217
incidents_070217.csv >> incidents_070217
csv files converted to points:
GoG123_20170205.csv >> GoG123_20170205
GoG123_20170206.csv >> GoG123_20170206
GoG123_20170207.csv >> GoG123_20170207


In [51]:
# Create Shipping tracks
#AIS123pts_test = AIS123pts[:1]
traffic_lyr = ais2traffic(FileGDB_path,AIS123pts,AISjoin_field,AISjoin_order,continent_polygon)

Output traffic files:
c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/working_dir/GoG123_20170205_offshore_traffic_lyr
['c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/working_dir/GoG123_20170205_offshore_traffic_lyr']
Output traffic files:
c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/working_dir/GoG123_20170206_offshore_traffic_lyr
['c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/working_dir/GoG123_20170205_offshore_traffic_lyr', 'c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/working_dir/GoG123_20170206_offshore_traffic_lyr']
Output traffic files:
c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/working_dir/GoG123_20170207_offshore_traffic_lyr
['c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/working_dir/GoG123_20170205_offshore_traffic_lyr', 'c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/working_dir/GoG123_20170206_offshore_traffic_lyr', 'c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/working_dir/GoG123_20170207_

In [60]:
# Create base grid
GRD = tes_grid(FileGDB_path, FileGDB_name, traffic_lyr, continent_polygon, grid_extent,spatial_ref, SIZE, SHAPE)

In [58]:
# Calculate incident density (km^-2). NEED TO ADD BOUNDARY INCIDENTS
inc_d, inc_date = incident_density(FileGDB_path, FileGDB_name, incidentspts, GRD)
#print(inc_d)
#print(inc_date)

c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/working_dir/risk_model_5-7.gdb/incident_density_050217
c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/working_dir/risk_model_5-7.gdb/incident_density_060217
c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/working_dir/risk_model_5-7.gdb/incident_density_070217


In [ ]:
# Calculate traffic density
Traf_dens, out_date = traffic_density(inc_d, traffic_lyr, GRD,FileGDB_path, FileGDB_name)

In [ ]:
inc_freq(Traf_dens, inc_d, GRD, FileGDB_path, FileGDB_name, "incident_frequency")

In [ ]:
# 1710
import datetime
print(datetime.datetime.now())

In [ ]:
#make feature layer for point shapefile  
for row in arcpy.SearchCursor(polygon shapefile):  
    #get FID and use it to construct a query that would only select the row (polygon) that you're looking at  
    #make feature layer using that query so now you have a one-feature feature layer  
    value = row.getValue("name of field you want a value from")  
    #use this feature layer to make a select by location operation on the point layer you made above  
    updaterows = arcpy.UpdateCursor(point feature layer) #the cursor should only contain the selected feature(s)  
    for rw in updaterows:  
        rw.setValue("field name you want to put the value in",value)  
        updaterows.UpdateRow(rw) 

In [ ]:
# Is this loop division useful, or will it be slow?

inFeature = outFeature
my_list = ['a', 'b', 'c', 'Shape_Area']
divfield = len(my_list) - 1

with arcpy.da.UpdateCursor(inFeature, my_list) as cursor:
    for row in cursor:
        for n in range(len(my_list)):
            row[n] = row[n] / row[divfield]
        cursor.updateRow(row)

In [30]:
copy GDB and rename inc_freq

join attributes inc_d and traf_c, and do cursor thing abouive to divide?
Can above be run with updates as feature is updated. DO we need a temp fiel of todays date to test if it has been run? 



#COPY FIELD, TAKE THOSE LESS THAN 10 m and = None
# Should we remove the short lines...? Will it be much quicker?
#10:46

#Find summary of these (how many)
i=0
#arcpy.AddField_management(FileGDB_path + FileGDB_name + "\\" + str(traffic[i])[-36:], "path_50m", "FLOAT","",9,"","","NULLABLE","REQUIRED","")
#arcpy.CalculateField_management(FileGDB_path + FileGDB_name + "\\" + str(traffic[i])[-36:],"path_50m", '!path_len! if !path_len! > 0.05 else None', 'PYTHON3')
#arcpy.SummarizeWithin_analysis (OUT,FileGDB_path + FileGDB_name + "\\" + str(traffic[i])[-36:],FileGDB_path + FileGDB_name + "\\" + "Traffic_density" + DATE, "KEEP_ALL", [['path_50m', 'SUM']], "ADD_SHAPE_SUM", "Kilometers","",NO_MIN_MAJ", "NO_PERCENT")

# ------------------
# THIS WONT WORK (ORDER)
arcpy.SelectLayerByAttribute_management(FileGDB_path + FileGDB_name + "\\" + str(traffic[i])[-36:]+"_test","NEW_SELECTION",'!path_len! <0.05')
arcpy.CopyFeatures_management (FileGDB_path + FileGDB_name + "\\" + str(traffic[i])[-36:], FileGDB_path + FileGDB_name + "\\" + str(traffic[i])[-36:]+"_test")
arcpy.DeleteFeatures_management(FileGDB_path + FileGDB_name + "\\" + str(traffic[i])[-36:]+"_test")
arcpy.SummarizeWithin_analysis (OUT,FileGDB_path + FileGDB_name + "\\" + str(traffic[i])[-36:]+"_test",FileGDB_path + FileGDB_name + "\\" + "Traffic_density" +"_"+str(traffic[i])[-29:-21], "KEEP_ALL", [['path_50m', 'SUM']], "ADD_SHAPE_SUM", "Kilometers","","NO_MIN_MAJ", "NO_PERCENT")


#arcpy.AddField_management(FileGDB_path + FileGDB_name + "\\" + str(traffic[i])[-36:], "path_50m", "FLOAT","",9,"","","NULLABLE","REQUIRED","")
#arcpy.CalculateField_management(FileGDB_path + FileGDB_name + "\\" + str(traffic[i])[-36:],"path_50m", '!path_len! if !path_len! > 0.05 else None', 'PYTHON3')
#arcpy.SummarizeWithin_analysis (OUT,FileGDB_path + FileGDB_name + "\\" + str(traffic[i])[-36:]+"_test",FileGDB_path + FileGDB_name + "\\" + "Traffic_density" + DATE, "KEEP_ALL", [['path_50m', 'SUM']], "ADD_SHAPE_SUM", "Kilometers",,"NO_MIN_MAJ", "NO_PERCENT")


ExecuteError: ERROR 000358: Invalid expression
Failed to execute (SelectLayerByAttribute).


In [14]:
import arcpy

# Set the workspace environment
arcpy.env.workspace ="c:/Users/Geollect/Documents/ArcGIS/Projects/risk_model/risk_data/continents/"

# Get a list of the feature classes in the input folder
feature_classes = arcpy.ListFeatureClasses()

# Loop through the list
for fc in feature_classes:
    # Create the spatial reference object
    spatial_ref = arcpy.Describe(fc).spatialReference

    # If the spatial reference is unknown
    if spatial_ref.name == "Unknown":
        print("{0} has an unknown spatial reference".format(fc))

    # Otherwise, print out the feature class name and
    # spatial reference
    else:
        print("{0} : {1}".format(fc, spatial_ref.name))

In [145]:
arcpy.TestSchemaLock (FileGDB_path + FileGDB_name + "/incidents_060217")

True